# Búsquedas aproximadas con PCA (Análisis de Componentes Principales)

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de junio de 2025 

### Gráficos interactivos

Para los gráficos se usa matplotlib:
```
pip install matplotlib
```

Para gráficos interactivos (por ej. hacer zoom):

  1. Instalar ipympl:  `pip install ipympl`
  2. Reiniciar jupyter 
  3. Reemplazar `%matplotlib inline` por `%matplotlib widget`


In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

## Descomentar esta linea para graficos interactivos
## %matplotlib widget


### Leer los datos

In [ ]:
import numpy


class Dataset:
    def __init__(self, nombre, archivo_q, archivo_r):
        self.nombre = nombre
        self.q = numpy.load(archivo_q)
        self.r = numpy.load(archivo_r)
        print("Dataset {}: Q={} R={}".format(self.nombre, self.q.shape, self.r.shape))


datasetA = Dataset("DATASET_A", "dataset_a_q.npy", "dataset_a_r.npy")
datasetB = Dataset("DATASET_B", "dataset_b_q.npy", "dataset_b_r.npy")

### Objeto que hace la comparación

In [ ]:
import time
import pyflann


class EvaluarPCA:
    def __init__(self, dataset):
        self.dataset = dataset
        self.flann = pyflann.FLANN()
        self.dimensiones = []
        self.varianzas = []
        self.precisiones = []
        self.eficiencias = []

    def linear_scan(self):
        print(
            "{} linear scan de Q={}xR={}".format(
                self.dataset.nombre, self.dataset.q.shape, self.dataset.r.shape
            )
        )
        self.flann.build_index(self.dataset.r, algorithm="linear")
        t0 = time.time()
        self.gt_nns, self.gt_dists = self.flann.nn_index(
            self.dataset.q, num_neighbors=1, cores=2
        )
        self.gt_segundos = time.time() - t0
        print(
            "{} linear scan = {:.1f} segundos".format(
                self.dataset.nombre, self.gt_segundos
            )
        )

    def calcular_PCA(self):
        self.promedios = self.dataset.r.mean(axis=0)
        datos_centrados = self.dataset.r - self.promedios
        # matriz de covarianza
        matriz_covarianza = numpy.cov(datos_centrados.transpose(), bias=True)
        # valores y vectores propios de la matriz de covarianza
        eigenvalues, eigenvectors = numpy.linalg.eig(matriz_covarianza)
        # indices ordenados
        indices_menor_a_mayor = eigenvalues.argsort()
        indices_mayor_a_menor = indices_menor_a_mayor[::-1]
        # guardar valores y vectores propios de mayor a menor
        self.eigenvalues = eigenvalues[indices_mayor_a_menor]
        self.eigenvectors = eigenvectors[:, indices_mayor_a_menor]

    def evaluar_busqueda(self, nns, dists, tiempo):
        correctas = 0
        incorrectas = 0
        for i in range(len(self.gt_nns)):
            if nns[i] == self.gt_nns[i]:
                correctas += 1
            else:
                incorrectas += 1
        precision = correctas / (correctas + incorrectas)
        eficiencia = tiempo / self.gt_segundos
        return precision, eficiencia

    def reducir_y_buscar(self, new_dims):
        varianza_retenida = numpy.sum(self.eigenvalues[:new_dims]) / numpy.sum(
            self.eigenvalues
        )
        dimension = new_dims / self.dataset.r.shape[1]
        # reducir R (parte de la fase offline)
        r_centrado = self.dataset.r - self.promedios
        transformacion = self.eigenvectors[:, :new_dims]
        r_newdim = r_centrado.dot(transformacion)
        # reducir Q (fase online)
        t0 = time.time()
        q_centrado = self.dataset.q - self.promedios
        q_newdim = q_centrado.dot(transformacion)
        # buscar entre Q y R reducidos con linear scan
        self.flann.build_index(r_newdim, algorithm="linear")
        nns_search, dists_search = self.flann.nn_index(
            q_newdim, num_neighbors=1, cores=2
        )
        segundos = time.time() - t0
        # medir resultado de la busqueda
        precision, eficiencia = self.evaluar_busqueda(
            nns_search, dists_search, segundos
        )
        # guardar resultados
        print(
            "{} PCA-{:<3} dim={:>6.1%}  var={:>6.1%}  precision={:>6.1%}  tiempo={:>6.1%} ({:4.1f} seg.)".format(
                self.dataset.nombre,
                new_dims,
                dimension,
                varianza_retenida,
                precision,
                eficiencia,
                segundos,
            )
        )
        self.dimensiones.append(dimension)
        self.varianzas.append(varianza_retenida)
        self.precisiones.append(precision)
        self.eficiencias.append(eficiencia)

# Experimento en ambos datasets (Dataset_A y Dataset_B)

El experimento consiste en:

  1. Calcular vecinos más cercanos con linear scan (linea base)
  2. Reducir dimensiones
  3. Calcular vecinos más cercanos en el espacio reducido
  4. Comparar los vecinos encontrados con los reales
  5. Calcular el % de respuestas correctas y el tiempo que tomó la búsqueda



In [ ]:
ev1 = EvaluarPCA(datasetA)
ev1.linear_scan()
ev1.calcular_PCA()
for dims in 2, 4, 8, 16, 24, 32, 48, 64, 80, 96, 112, 128:
    ev1.reducir_y_buscar(dims)

print()

ev2 = EvaluarPCA(datasetB)
ev2.linear_scan()
ev2.calcular_PCA()
for dims in 2, 4, 8, 16, 24, 32, 48, 64, 80, 96, 112, 128:
    ev2.reducir_y_buscar(dims)

### Graficar

In [ ]:
plt.plot(
    ev1.precisiones,
    ev1.eficiencias,
    label=ev1.dataset.nombre,
    color="r",
    linestyle="--",
    marker="o",
    markerfacecolor="c",
    markersize=8,
)
plt.plot(
    ev2.precisiones,
    ev2.eficiencias,
    label=ev2.dataset.nombre,
    color="g",
    linestyle="-.",
    marker="^",
    markerfacecolor="m",
    markersize=8,
)
plt.xlabel("Precisión NN [comparado con LS]")
plt.ylabel("Tiempos [comparado con LS]")
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.legend()
plt.show()

En mi computador, el linear scan en cada conjunto toma 9.2 segundos

Cuando los conjuntos se reducen a 32 dimensiones, el tiempo de búsqueda es cercano a la mitad del tiempo original. 

```
DATASET_A PCA-32  dim= 25.0%  var= 82.0%  precision= 64.1%  tiempo= 49.3% ( 4.6 seg.)
DATASET_B PCA-32  dim= 25.0%  var= 88.1%  precision= 54.8%  tiempo= 47.4% ( 4.4 seg.)
```

La calidad de respuesta obtenida es que entre 55% a 65% de vecinos más cercanos correctos.


**¿Cómo es la relación precision vs eficiencia comparado con los árboles y con LSH? (ver anexos anteriores)**



**Pregunta:** ¿Sería posible estimar la calidad de la respuesta de la búsqueda aproximada sin tener que realizar el linear scan? Notar que en DATASET_B para 32 dimensiones la varianza retenida en la proyección es mayor que en DATASET_A, pero la calidad de respuesta es peor (menos NN correctos)
